## neo4j-contrib:neo4j-spark-connector versions
https://mvnrepository.com/artifact/neo4j-contrib/neo4j-spark-connector

In [1]:
%%init_spark
# Configure Spark to use a local master
launcher.master = "local[*]"

# needs java 1.8
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.4.0-M6"

# needs java 1.8
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.2.1-M5"

# saveGraph mixes up id and number
launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.1.0-M4"

# saveGraph mixes up id and number
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.1.0-M3"

# doesnt have neo.saveGraph()
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.0.0-M2" 

# doesnt do neo = Neo4j(sc)
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.0.0-M1"  

In [2]:
println(sc.master)
// val rdd = sc.parallelize(0 to 999)
// rdd.takeSample(false, 5)

Intitializing Scala interpreter ...

Spark Web UI available at http://172.18.0.7:4040
SparkContext available as 'sc' (version = 2.0.0, master = local[*], app id = local-1551949449714)
SparkSession available as 'spark'


local[*]


In [3]:
import org.neo4j.spark._

val neo = Neo4j(sc)

import org.apache.spark.graphx._
import org.apache.spark.graphx.lib._

import org.neo4j.spark._
neo: org.neo4j.spark.Neo4j = org.neo4j.spark.Neo4j@3ee7e8cb
import org.apache.spark.graphx._
import org.apache.spark.graphx.lib._


In [4]:
// // load graph via Cypher query
// // val graphQuery = "MATCH (n:Person)-[r:KNOWS]->(m:Person) RETURN id(n) as source, id(m) as target, type(r) as value SKIP {_skip} LIMIT {_limit}"
// // val graph: Graph[Long, String] = neo.rels(graphQuery).partitions(7).batch(200).loadGraph
// val graphQuery = "MATCH (n:Person)-[r:KNOWS]->(m:Person) RETURN id(n) as source, id(m) as target, type(r) as value"
// val graph: Graph[Long, String] = neo.rels(graphQuery).partitions(1).batch(200).loadGraph
// println(graph.vertices.count)
// println(graph.edges.count)

In [5]:
val mypattern = neo.pattern(("Person","id"),("KNOWS","years"),("Person","id"))
val graph2 = mypattern.partitions(2).batch(15).loadGraph[Long,Long]

mypattern: org.neo4j.spark.Neo4j = org.neo4j.spark.Neo4j@3ee7e8cb
graph2: org.apache.spark.graphx.Graph[Long,Long] = org.apache.spark.graphx.impl.GraphImpl@7a06b483


In [6]:
println(graph2.vertices.count)
println(graph2.edges.count)

10
20


In [7]:
// val graph2_pagerank = PageRank.run(graph2,500)
val graph2_pagerank = PageRank.run(graph2,10)

graph2_pagerank: org.apache.spark.graphx.Graph[Double,Double] = org.apache.spark.graphx.impl.GraphImpl@3e9cc541


In [8]:
graph2.edges.collect()

res3: Array[org.apache.spark.graphx.Edge[Long]] = Array(Edge(53,55,2), Edge(53,56,3), Edge(54,56,2), Edge(54,57,3), Edge(55,57,2), Edge(57,80,3), Edge(58,81,3), Edge(59,81,2), Edge(59,82,3), Edge(55,58,3), Edge(56,58,2), Edge(56,59,3), Edge(57,59,2), Edge(58,80,2), Edge(80,53,7), Edge(80,82,2), Edge(81,53,8), Edge(81,54,7), Edge(82,54,8), Edge(82,55,7))


In [9]:
graph2.vertices.collect()

res4: Array[(org.apache.spark.graphx.VertexId, Long)] = Array((56,0), (80,0), (81,0), (53,0), (57,0), (82,0), (54,0), (58,0), (55,0), (59,0))


In [10]:
graph2_pagerank.edges.collect()

res5: Array[org.apache.spark.graphx.Edge[Double]] = Array(Edge(53,55,0.5), Edge(53,56,0.5), Edge(54,56,0.5), Edge(54,57,0.5), Edge(55,57,0.5), Edge(57,80,0.5), Edge(58,81,0.5), Edge(59,81,0.5), Edge(59,82,0.5), Edge(55,58,0.5), Edge(56,58,0.5), Edge(56,59,0.5), Edge(57,59,0.5), Edge(58,80,0.5), Edge(80,53,0.5), Edge(80,82,0.5), Edge(81,53,0.5), Edge(81,54,0.5), Edge(82,54,0.5), Edge(82,55,0.5))


In [11]:
graph2_pagerank.vertices.collect()

res6: Array[(org.apache.spark.graphx.VertexId, Double)] = Array((56,0.8326567563103858), (80,0.8326567563103858), (81,0.8326567563103858), (53,0.8326567563103858), (57,0.8326567563103858), (82,0.8326567563103858), (54,0.8326567563103858), (58,0.8326567563103858), (55,0.8326567563103858), (59,0.8326567563103858))


In [12]:
graph2.vertices.sortBy(_._2).take(25)

res7: Array[(org.apache.spark.graphx.VertexId, Long)] = Array((56,0), (80,0), (81,0), (53,0), (57,0), (82,0), (54,0), (58,0), (55,0), (59,0))


In [13]:
-- Stuff below doesnt work correctly

<console>: 35: error: not found: value --

In [ ]:
val mypattern_cmon = neo.pattern("Person",Seq("KNOWS"),"Person")

In [ ]:
// uses pattern from above to save the data, merge parameter is false by default, only update existing nodes
neo.saveGraph(graph2_pagerank, "rankcmon", mypattern_cmon.pattern, merge = true)

In [ ]:
// uses pattern from above to save the data, merge parameter is false by default, only update existing nodes
neo.saveGraph(graph2_pagerank, "rankcmon")

In [ ]:
// uses pattern from parameter to save the data, merge = true also create new nodes and relationships
neo.saveGraph(graph, "rank",neo.pattern(("Person","id"),("FRIEND","years"),("Person","id")), merge = true)